In [2]:
API_TOKEN = '' # Define your API token
IB_CONTEXT = '' # Define organization ID to use organization account, define user ID to use community account
CHATBOT_ID = ""

In [3]:
user_q = "What is the best way to prepare my broccoli farm for a drought?"  ## User input goes here

api_response = ""

prompt_eng = f"Weather data: {api_response}\nFarmer's question: {user_q}"  ##TODO: engineer this better

payload = {
  "query": prompt_eng,
  "source_app": {
    "type": "CHATBOT",
    "id": CHATBOT_ID
  },
  "model": "multistep",
  "include_source_info": True
}

In [17]:
import time, json
from typing import Dict, Text, Tuple, Any
import requests
from requests import Response

API_BASE_URL = 'https://aihub.instabase.com/api' # Edit root API URL if your organization has a custom domain
url = API_BASE_URL + '/v2/queries'

API_HEADERS = {
    'Authorization': f'Bearer {API_TOKEN}',
    'IB-Context': f'{IB_CONTEXT}'
}

def query_chatbot(data: Dict[Text, Any]) -> Response:
  print('Querying chatbot')
  response = requests.post(url=url, headers=API_HEADERS, data=json.dumps(payload))
  return response

def get_query_status(query_id: Text) -> Tuple[Dict, Text]:
  print(f'Getting query request status for {query_id}')
  while True:
    response = requests.get(f'{url}/{query_id}', headers=API_HEADERS)
    if not response.ok:
        return None, f'Error while getting the status for query request :  {response.status_code} - {response.text}'

    response_json = response.json()
    status = response_json.get('status')
    if not status:
        return None, f'Getting the status for query request : {response_json}'

    # If status of query request is COMPLETE stop polling and return
    if status in ['COMPLETE']:
      return response_json, None

    print('Query still processing, please wait!')

    # If it is RUNNING, the script waits for 5 seconds before the next poll
    time.sleep(5)


## Handled with the `payload` var

data: Dict[Text, Any] = {}
data['query'] = 'What is the best way to prepare my broccoli farm for a drought?' # Define your chatbot query
data['model_name'] = 'multistep' # Define model to use, 'multistep-lite' (multistep model) or 'multistep' (research mode)
data['include_source_info'] = True # Set to 'true' to get information about source documents referenced when generating response
data['source_app'] = {'type':'CHATBOT', 'id':'awef'} # Define chatbot ID

In [18]:
# Query the chatbot
response = query_chatbot(data=payload)
query_id = None
if response.ok:
  resp_data = response.json()
  query_id = resp_data.get('query_id')
  if not query_id:
    print('Unable to query the chatbot.')

  print(f'Response from query chatbot : {resp_data}')
else:
  print(f'Unable to connect to the chatbot query API - {response}.')
  print(f'Response status code: {response.status_code}')
  print(f'Response content: {response.text}')

Querying chatbot
Response from query chatbot : {'query_id': '01925efc-de86-7a2b-b8ad-c7fb68365b9c'}


In [19]:
# Get the status of query request
if query_id:
  response_json, err = get_query_status(query_id=query_id)
  if err:
    print(f'Error occurred: {err}')
  else:
    print(f'Query request status : {response_json}')

Getting query request status for 01925efc-de86-7a2b-b8ad-c7fb68365b9c
Query still processing, please wait!
Query still processing, please wait!
Query still processing, please wait!
Query request status : {'query_id': '01925efc-de86-7a2b-b8ad-c7fb68365b9c', 'status': 'COMPLETE', 'results': [{'response': 'To prepare your broccoli farm for a drought, consider the following strategies based on the provided documents:\n\n1. **Soil Improvement**:\n   - **Healthy Soils**: Focus on building healthy soils with good and stable aggregation, enhanced organic matter levels, and limited or no compaction. This can help in "drought-proofing" your farm by improving water infiltration and reducing evaporation losses from the soil.\n   - **Reduced Tillage**: Implement reduced tillage practices with residues on the surface to enhance water infiltration and reduce evaporation.\n   - **Cover Crops**: Use cover crops to act as a water-conserving surface mulch once they are suppressed. Although cover crops us